# GE Advance vs Random Scanner Configuration 

In [1]:
import yaml
import math
import pandas as pd
from pathlib import Path
from pyprojroot import here

In [2]:
CFG_PATH = here().joinpath("config_comparison")
GE_DISCOVERY = CFG_PATH.joinpath("discovery_st.yaml")
RANDOM = CFG_PATH.joinpath("scanner_2.yaml")
CONFIGS = {
    GE_DISCOVERY.stem: GE_DISCOVERY,
    RANDOM.stem: RANDOM
}

In [3]:
RECON_PARAMS = [
    "scanner_name",
    "average_doi",
    "scanner_radius",
    "num_rings",
    "axial_fov",
    "z_crystal_size",
    "transaxial_crystal_size",
    "crystal_thickness",
    "num_aa_bins",
    "num_td_bins",
    "zoomFactor",
    "numberOfSubsets",
    "numberOfIterations",
    "xyOutputSize"
]

In [4]:
for name, path_ in CONFIGS.items():
    with open(path_, 'r') as cfg:
        CONFIGS[name] = yaml.safe_load(cfg)

In [5]:
df = pd.DataFrame(CONFIGS.values())[RECON_PARAMS]
df

,scanner_name,average_doi,scanner_radius,num_rings,axial_fov,z_crystal_size,transaxial_crystal_size,crystal_thickness,num_aa_bins,num_td_bins,zoomFactor,numberOfSubsets,numberOfIterations,xyOutputSize
0,GE Discovery ST,0.84,44.310,24,15.700,0.630,0.630,3.000,210,249,1.55,7,32,128
1,Scanner 2,1.20,43.732,36,24.766,0.671,0.443,2.938,310,163,3.50,5,70,320


In [6]:
df["n_transaxial_detectors"] = 2 * df["num_aa_bins"]
df["scanner_perimeter_radius"] = 2 * math.pi * df["scanner_radius"]
df["scanner_perimeter_transaxial_detectors"] = df["n_transaxial_detectors"] * df["transaxial_crystal_size"]
df["angle"] = 2 * math.pi / (2 * df["n_transaxial_detectors"])
df["scanner_perimeter_geometric"] = 2 * df["scanner_radius"] * df["n_transaxial_detectors"] * df["angle"]


In [7]:
perimeter_cols = [col for col in df.columns if col.startswith("scanner_perimeter")] + ["scanner_name"]
df[perimeter_cols]

,scanner_perimeter_radius,scanner_perimeter_transaxial_detectors,scanner_perimeter_geometric,scanner_name
0,278.407941,264.60,278.407941,GE Discovery ST
1,274.776260,274.66,274.776260,Scanner 2


In [23]:
cols_ofi = ["scanner_name", "bin_size", "transaxial_crystal_size", "scanner_radius", "bins_length", "bin_size"]
bins_cols = [col for col in df.columns if col.startswith("num_")] + cols_ofi
df["bin_size"] = (2 * df["scanner_radius"]) / df["num_td_bins"]
df["bins_length"] = df["transaxial_crystal_size"] * df["num_td_bins"]
df[bins_cols]

,num_rings,num_aa_bins,num_td_bins,scanner_name,bin_size,transaxial_crystal_size,scanner_radius,bins_length,bin_size
0,24,210,249,GE Discovery ST,0.355904,0.630,44.310,156.870,0.355904
1,36,310,163,Scanner 2,0.536589,0.443,43.732,72.209,0.536589
